In [3]:
# 加载并标注数据到内存
import os
import sys
import numpy as np
import cv2

IMAGE_SIZE = 64

In [4]:
# 按照指定图像大小调整尺寸
# 判断图片是不是四边等长，也就是图片是不是正方形。如果不是，则短的那两边增加两条黑色的边框，使图像变成正方形
def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0, 0, 0, 0)
    
    #获取图像尺寸
    h, w, _ = image.shape
    
    #对于长宽不相等的图片，找到最长的一边
    longest_edge = max(h, w)    
    
    #计算短边需要增加多上像素宽度使其与长边等长
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    
    #RGB颜色
    BLACK = [0, 0, 0]
    
    #给图像增加边界，是图片长、宽等长，cv2.BORDER_CONSTANT指定边界颜色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    
    #调整图像大小并返回
    return cv2.resize(constant, (height, width))

In [5]:
# 读取训练数据
images = []
labels = []
def read_path(path_name):    
    for dir_item in os.listdir(path_name):
        # 从初始路径开始叠加，合并成可识别的操作路径
        full_path = os.path.join(path_name, dir_item)
        if os.path.isdir(full_path):    #如果是文件夹，继续递归调用
            read_path(full_path)
        else:   #文件
            if dir_item.endswith('.jpg'):
                image = cv2.imread(full_path)
                print(full_path)
                try:
                    image = resize_image(image, IMAGE_SIZE, IMAGE_SIZE)
                except AttributeError:
                    print("shape not found")

                # 放开这个代码，可以看到resize_image()函数的实际调用效果
                #cv2.imwrite('1.jpg', image)
                
                images.append(image)                
                labels.append(path_name)                                
                    
    return images,labels

In [6]:
# 从指定路径读取训练数据
def load_dataset(path_name):
    images,labels = read_path(path_name)    
    
    #将输入的所有图片转成四维数组，尺寸为(图片数量*IMAGE_SIZE*IMAGE_SIZE*3)
    #我和闺女两个人共1200张图片，IMAGE_SIZE为64，故对我来说尺寸为1200 * 64 * 64 * 3
    #图片为64 * 64像素,一个像素3个颜色值(RGB)
    images = np.array(images)
    print(images.shape)    
    
    #标注数据，'user'文件夹下都是我的脸部图像，全部指定为0，另外一个文件夹下是闺女的，全部指定为1
    labels = np.array([0 if label.endswith('user') else 1 for label in labels])    
    
    return images, labels

In [7]:
#imagesPath = "data"
#images, labels = load_dataset(imagesPath)

In [8]:
import random

import numpy as np
from sklearn.cross_validation import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [9]:
# 于数据加载及预处理
class Dataset:
    def __init__(self, path_name):
        #训练集
        self.train_images = None
        self.train_labels = None
        
        #验证集
        self.valid_images = None
        self.valid_labels = None
        
        #测试集
        self.test_images  = None            
        self.test_labels  = None
        
        #数据集加载路径
        self.path_name    = path_name
        
        #当前库采用的维度顺序
        self.input_shape = None
        
    #加载数据集并按照交叉验证的原则划分数据集并进行相关预处理工作
    def load(self, img_rows = IMAGE_SIZE, img_cols = IMAGE_SIZE, 
             img_channels = 3, nb_classes = 2):
        #加载数据集到内存
        images, labels = load_dataset(self.path_name)        
        
        train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, test_size = 0.3, random_state = random.randint(0, 100))        
        _, test_images, _, test_labels = train_test_split(images, labels, test_size = 0.5, random_state = random.randint(0, 100))                
        
        #当前的维度顺序如果为'th'，则输入图片数据时的顺序为：channels,rows,cols，否则:rows,cols,channels
        #这部分代码就是根据keras库要求的维度顺序重组训练数据集
        if K.image_dim_ordering() == 'th':
            train_images = train_images.reshape(train_images.shape[0], img_channels, img_rows, img_cols)
            valid_images = valid_images.reshape(valid_images.shape[0], img_channels, img_rows, img_cols)
            test_images = test_images.reshape(test_images.shape[0], img_channels, img_rows, img_cols)
            self.input_shape = (img_channels, img_rows, img_cols)            
        else:
            train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, img_channels)
            valid_images = valid_images.reshape(valid_images.shape[0], img_rows, img_cols, img_channels)
            test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, img_channels)
            self.input_shape = (img_rows, img_cols, img_channels)            
            
            #输出训练集、验证集、测试集的数量
            print(train_images.shape[0], 'train samples')
            print(valid_images.shape[0], 'valid samples')
            print(test_images.shape[0], 'test samples')
        
            #我们的模型使用categorical_crossentropy作为损失函数，因此需要根据类别数量nb_classes将
            #类别标签进行one-hot编码使其向量化，在这里我们的类别只有两种，经过转化后标签数据变为二维
            train_labels = np_utils.to_categorical(train_labels, nb_classes)                        
            valid_labels = np_utils.to_categorical(valid_labels, nb_classes)            
            test_labels = np_utils.to_categorical(test_labels, nb_classes)                        
        
            #像素数据浮点化以便归一化
            train_images = train_images.astype('float32')            
            valid_images = valid_images.astype('float32')
            test_images = test_images.astype('float32')
            
            #将其归一化,图像的各像素值归一化到0~1区间
            train_images /= 255
            valid_images /= 255
            test_images /= 255            
        
            self.train_images = train_images
            self.valid_images = valid_images
            self.test_images  = test_images
            self.train_labels = train_labels
            self.valid_labels = valid_labels
            self.test_labels  = test_labels

In [10]:
# CNN网络模型类            
class Model:
    def __init__(self):
        self.model = None 
        
    # 建立模型
    def build_model(self, dataset, nb_classes = 2):
        #构建一个空的网络模型，它是一个线性堆叠模型，各神经网络层会被顺序添加，专业名称为序贯模型或线性堆叠模型
        self.model = Sequential() 
        
        #以下代码将顺序添加CNN网络需要的各层，一个add就是一个网络层
        self.model.add(Conv2D(32, 3, 3, border_mode='same', 
                                     input_shape = dataset.input_shape))    #1 2维卷积层
        self.model.add(Activation('relu'))                                  #2 激活函数层
        
        self.model.add(Conv2D(32, 3, 3))                             #3 2维卷积层                             
        self.model.add(Activation('relu'))                                  #4 激活函数层
        
        self.model.add(MaxPooling2D(pool_size=(2, 2)))                      #5 池化层
        self.model.add(Dropout(0.25))                                       #6 Dropout层

        self.model.add(Conv2D(64, 3, 3, border_mode='same'))         #7  2维卷积层
        self.model.add(Activation('relu'))                                  #8  激活函数层
        
        self.model.add(Conv2D(64, 3, 3))                             #9  2维卷积层
        self.model.add(Activation('relu'))                                  #10 激活函数层
        
        self.model.add(MaxPooling2D(pool_size=(2, 2)))                      #11 池化层
        self.model.add(Dropout(0.25))                                       #12 Dropout层

        self.model.add(Flatten())                                           #13 Flatten层
        self.model.add(Dense(512))                                          #14 Dense层,又被称作全连接层
        self.model.add(Activation('relu'))                                  #15 激活函数层   
        self.model.add(Dropout(0.5))                                        #16 Dropout层
        self.model.add(Dense(nb_classes))                                   #17 Dense层
        self.model.add(Activation('softmax'))                               #18 分类层，输出最终结果
        
        #输出模型概况
        self.model.summary()
        
        
    #训练模型
    def train(self, dataset, batch_size = 20, nb_epoch = 10, data_augmentation = True):        
        sgd = SGD(lr = 0.01, decay = 1e-6, 
                  momentum = 0.9, nesterov = True) #采用SGD+momentum的优化器进行训练，首先生成一个优化器对象  
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=sgd,
                           metrics=['accuracy'])   #完成实际的模型配置工作
        
        #不使用数据提升，所谓的提升就是从我们提供的训练数据中利用旋转、翻转、加噪声等方法创造新的
        #训练数据，有意识的提升训练数据规模，增加模型训练量
        if not data_augmentation:            
            self.model.fit(dataset.train_images,
                           dataset.train_labels,
                           batch_size = batch_size,
                           nb_epoch = nb_epoch,
                           validation_data = (dataset.valid_images, dataset.valid_labels),
                           shuffle = True)
        #使用实时数据提升
        else:            
            #定义数据生成器用于数据提升，其返回一个生成器对象datagen，datagen每被调用一
            #次其生成一组数据（顺序生成），节省内存，其实就是python的数据生成器
            datagen = ImageDataGenerator(
                featurewise_center = False,             #是否使输入数据去中心化（均值为0），
                samplewise_center  = False,             #是否使输入数据的每个样本均值为0
                featurewise_std_normalization = False,  #是否数据标准化（输入数据除以数据集的标准差）
                samplewise_std_normalization  = False,  #是否将每个样本数据除以自身的标准差
                zca_whitening = False,                  #是否对输入数据施以ZCA白化
                rotation_range = 20,                    #数据提升时图片随机转动的角度(范围为0～180)
                width_shift_range  = 0.2,               #数据提升时图片水平偏移的幅度（单位为图片宽度的占比，0~1之间的浮点数）
                height_shift_range = 0.2,               #同上，只不过这里是垂直
                horizontal_flip = True,                 #是否进行随机水平翻转
                vertical_flip = False)                  #是否进行随机垂直翻转

            #计算整个训练样本集的数量以用于特征值归一化、ZCA白化等处理
            datagen.fit(dataset.train_images)                        

            #利用生成器开始训练模型
            self.model.fit_generator(datagen.flow(dataset.train_images, dataset.train_labels,
                                                   batch_size = batch_size),
                                     samples_per_epoch = dataset.train_images.shape[0],
                                     nb_epoch = nb_epoch,
                                     validation_data = (dataset.valid_images, dataset.valid_labels))
            
    MODEL_PATH = './user.face.model.h5'
    # 保存模型
    def save_model(self, file_path = MODEL_PATH):
        self.model.save(file_path)
    
    # 读取模型
    def load_model(self, file_path = MODEL_PATH):
        self.model = load_model(file_path)
        
    # 评估模型
    def evaluate(self, dataset):
        score = self.model.evaluate(dataset.test_images, dataset.test_labels, verbose = 1)
        print("%s: %.2f%%" % (self.model.metrics_names[1], score[1] * 100))
        
    # 识别人脸
    def face_predict(self, image):    
        # 依然是根据后端系统确定维度顺序
        if K.image_dim_ordering() == 'th' and image.shape != (1, 3, IMAGE_SIZE, IMAGE_SIZE):
            image = resize_image(image)                             #尺寸必须与训练集一致都应该是IMAGE_SIZE x IMAGE_SIZE
            image = image.reshape((1, 3, IMAGE_SIZE, IMAGE_SIZE))   #与模型训练不同，这次只是针对1张图片进行预测    
        elif K.image_dim_ordering() == 'tf' and image.shape != (1, IMAGE_SIZE, IMAGE_SIZE, 3):
            image = resize_image(image)
            image = image.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 3))                    
        
        #浮点并归一化
        image = image.astype('float32')
        image /= 255
        
        #给出输入属于各个类别的概率，我们是二值类别，则该函数会给出输入图像属于0和1的概率各为多少
        result = self.model.predict_proba(image)
        print('result:', result)
        
        #给出类别预测：0或者1
        result = self.model.predict_classes(image)        

        #返回类别预测结果
        return result[0]

In [81]:
dataset = Dataset("data")    
dataset.load()

data\other\0.jpg
data\other\1.jpg
data\other\10.jpg
data\other\100.jpg
data\other\1003.jpg
data\other\101.jpg
data\other\102.jpg
data\other\103.jpg
data\other\104.jpg
data\other\1044.jpg
data\other\105.jpg
data\other\106.jpg
data\other\1061.jpg
data\other\107.jpg
data\other\108.jpg
data\other\109.jpg
data\other\1097.jpg
data\other\11.jpg
data\other\110.jpg
data\other\111.jpg
data\other\112.jpg
data\other\113.jpg
data\other\114.jpg
data\other\115.jpg
data\other\116.jpg
data\other\117.jpg
data\other\118.jpg
data\other\119.jpg
data\other\12.jpg
data\other\120.jpg
data\other\122.jpg
data\other\123.jpg
data\other\124.jpg
data\other\125.jpg
data\other\126.jpg
data\other\127.jpg
data\other\128.jpg
data\other\129.jpg
data\other\13.jpg
data\other\130.jpg
data\other\131.jpg
data\other\132.jpg
data\other\133.jpg
data\other\134.jpg
data\other\135.jpg
data\other\136.jpg
data\other\137.jpg
data\other\138.jpg
data\other\139.jpg
data\other\14.jpg
data\other\140.jpg
data\other\141.jpg
data\other\142.jp

data\other\835.jpg
data\other\836.jpg
data\other\837.jpg
data\other\838.jpg
data\other\84.jpg
data\other\840.jpg
data\other\842.jpg
data\other\845.jpg
data\other\847.jpg
data\other\85.jpg
data\other\850.jpg
data\other\852.jpg
data\other\853.jpg
data\other\854.jpg
data\other\855.jpg
data\other\856.jpg
data\other\86.jpg
data\other\860.jpg
data\other\861.jpg
data\other\864.jpg
data\other\866.jpg
data\other\87.jpg
data\other\88.jpg
data\other\89.jpg
data\other\9.jpg
data\other\90.jpg
data\other\91.jpg
data\other\92.jpg
data\other\93.jpg
data\other\936.jpg
data\other\94.jpg
data\other\949.jpg
data\other\95.jpg
data\other\96.jpg
data\other\967.jpg
data\other\97.jpg
data\other\98.jpg
data\other\99.jpg
data\other\993.jpg
data\user\100.jpg
data\user\1000.jpg
data\user\1001.jpg
data\user\1002.jpg
data\user\1003.jpg
data\user\1004.jpg
data\user\1005.jpg
data\user\1006.jpg
data\user\1007.jpg
data\user\1008.jpg
data\user\1009.jpg
data\user\101.jpg
data\user\1010.jpg
data\user\1011.jpg
data\user\101

data\user\190.jpg
data\user\191.jpg
data\user\192.jpg
data\user\193.jpg
data\user\194.jpg
data\user\195.jpg
data\user\196.jpg
data\user\197.jpg
data\user\198.jpg
data\user\199.jpg
data\user\200.jpg
data\user\201.jpg
data\user\202.jpg
data\user\203.jpg
data\user\204.jpg
data\user\205.jpg
data\user\206.jpg
data\user\207.jpg
data\user\208.jpg
data\user\209.jpg
data\user\21.jpg
data\user\210.jpg
data\user\211.jpg
data\user\212.jpg
data\user\213.jpg
data\user\214.jpg
data\user\215.jpg
data\user\216.jpg
data\user\217.jpg
data\user\218.jpg
data\user\219.jpg
data\user\22.jpg
data\user\220.jpg
data\user\221.jpg
data\user\222.jpg
data\user\223.jpg
data\user\224.jpg
data\user\225.jpg
data\user\226.jpg
data\user\227.jpg
data\user\228.jpg
data\user\229.jpg
data\user\230.jpg
data\user\231.jpg
data\user\232.jpg
data\user\233.jpg
data\user\234.jpg
data\user\235.jpg
data\user\236.jpg
data\user\237.jpg
data\user\238.jpg
data\user\239.jpg
data\user\24.jpg
data\user\240.jpg
data\user\241.jpg
data\user\242

data\user\78.jpg
data\user\780.jpg
data\user\781.jpg
data\user\782.jpg
data\user\783.jpg
data\user\784.jpg
data\user\785.jpg
data\user\79.jpg
data\user\80.jpg
data\user\81.jpg
data\user\82.jpg
data\user\83.jpg
data\user\84.jpg
data\user\85.jpg
data\user\86.jpg
data\user\87.jpg
data\user\88.jpg
data\user\89.jpg
data\user\90.jpg
data\user\91.jpg
data\user\919.jpg
data\user\92.jpg
data\user\920.jpg
data\user\921.jpg
data\user\922.jpg
data\user\923.jpg
data\user\924.jpg
data\user\925.jpg
data\user\926.jpg
data\user\927.jpg
data\user\928.jpg
data\user\929.jpg
data\user\93.jpg
data\user\930.jpg
data\user\931.jpg
data\user\932.jpg
data\user\933.jpg
data\user\934.jpg
data\user\935.jpg
data\user\936.jpg
data\user\937.jpg
data\user\938.jpg
data\user\939.jpg
data\user\94.jpg
data\user\940.jpg
data\user\941.jpg
data\user\942.jpg
data\user\943.jpg
data\user\944.jpg
data\user\945.jpg
data\user\946.jpg
data\user\947.jpg
data\user\948.jpg
data\user\949.jpg
data\user\95.jpg
data\user\950.jpg
data\user\

In [82]:
model = Model()
model.build_model(dataset)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
  del sys.path[0]
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  app.launch_new_instance()
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_19 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 62, 62, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 31, 31, 64)        18496     
__________

In [83]:
#测试训练函数的代码
model.train(dataset)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., steps_per_epoch=162, epochs=10)`


Epoch 1/10
162/162 [==============================] - 133s 824ms/step - loss: 0.2809 - acc: 0.8861 - val_loss: 0.0219 - val_acc: 0.9978
Epoch 2/10
162/162 [==============================] - 136s 842ms/step - loss: 0.0448 - acc: 0.9843 - val_loss: 0.0147 - val_acc: 0.9964
Epoch 3/10
162/162 [==============================] - 133s 819ms/step - loss: 0.0364 - acc: 0.9904 - val_loss: 0.0068 - val_acc: 0.9986
Epoch 4/10
162/162 [==============================] - 133s 819ms/step - loss: 0.0253 - acc: 0.9932 - val_loss: 0.0086 - val_acc: 0.9986
Epoch 5/10
162/162 [==============================] - 132s 813ms/step - loss: 0.0164 - acc: 0.9963 - val_loss: 0.0844 - val_acc: 0.9641
Epoch 6/10
162/162 [==============================] - 132s 817ms/step - loss: 0.0264 - acc: 0.9920 - val_loss: 0.0054 - val_acc: 0.9986
Epoch 7/10
162/162 [==============================] - 139s 857ms/step - loss: 0.0231 - acc: 0.9957 - val_loss: 0.0043 - val_acc: 0.9986
Epoch 8/10
162/162 [============================

In [84]:
model.save_model(file_path = './model/user.face.model.h5')

In [88]:
# 评估模型
model = Model()
model.load_model(file_path = './model/user.face.model.h5')
model.evaluate(dataset)

2322/2322 [==============================] - 26s 11ms/step
acc: 99.87%


In [11]:
import gc

In [15]:
def CatchUsbVideo(camera_idx, file_path):
    # 视频来源
    cap = cv2.VideoCapture(camera_idx)  
    
    # 加载模型
    model = Model()
    model.load_model(file_path)    
    
    # 告诉OpenCV使用人脸识别分类器
    classfier = cv2.CascadeClassifier("../data/haarcascade_frontalface_alt2.xml")  
    
    # 框住人脸的矩形边框颜色       
    color = (0, 255, 0)
    
    # 循环检测识别人脸
    while True:
        _, frame = cap.read()   # 读取一帧视频
        
        # 图像灰化，降低计算复杂度
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)           

        #利用分类器识别出哪个区域为人脸
        faceRects = classfier.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 2, minSize = (64, 64))
        #大于0则检测到人脸     
        if len(faceRects) > 0: 
            for faceRect in faceRects: # 单独框出每一张人脸
                x, y, w, h = faceRect
                
                # 截取脸部图像提交给模型识别这是谁
                image = frame[y - 10: y + h + 10, x - 10: x + w + 10]
                faceID = model.face_predict(image)   
                
                #如果是“新坦结衣”
                if faceID == 0:                                                        
                    cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                    
                    #文字提示是谁
                    cv2.putText(frame,'XTJY', 
                                (x + 30, y + 30),                      #坐标
                                cv2.FONT_HERSHEY_SIMPLEX,              #字体
                                1,                                     #字号
                                (255,0,255),                           #颜色
                                2)                                     #字的线宽
                if faceID == 1:                                                        
                    cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                    
                    #文字提示是谁
                    cv2.putText(frame,'AAA', 
                                (x + 30, y + 30),                      #坐标
                                cv2.FONT_HERSHEY_SIMPLEX,              #字体
                                1,                                     #字号
                                (255,0,255),                           #颜色
                                2)                                     #字的线宽
                            
        cv2.imshow("识别新坦结衣", frame)
        
        #等待10毫秒看是否有按键输入
        k = cv2.waitKey(10)
        #如果输入q则退出循环
        if k & 0xFF == ord('q'):
            break

    #释放摄像头并销毁所有窗口
    cap.release()
    cv2.destroyAllWindows()

In [16]:
# 使用训练好的模型的参数
file_path = './model/user.face.model.h5'
# 人脸识别分类器本地存储路径
camera = "新垣结衣.mp4"

CatchUsbVideo(camera, file_path)

result: [[  9.99408603e-01   5.91425283e-04]]
result: [[  9.99457061e-01   5.42947499e-04]]
result: [[  9.99067008e-01   9.32979048e-04]]
result: [[  9.99301553e-01   6.98452990e-04]]
result: [[  9.99573767e-01   4.26234881e-04]]
result: [[  9.99748051e-01   2.51914928e-04]]
result: [[  9.99660373e-01   3.39575374e-04]]
result: [[  9.99668002e-01   3.32045398e-04]]
result: [[  9.99666095e-01   3.33859760e-04]]
result: [[  9.99607503e-01   3.92517832e-04]]
result: [[  9.99632597e-01   3.67401924e-04]]
result: [[  9.99646068e-01   3.53884476e-04]]
result: [[  9.99669433e-01   3.30529379e-04]]
result: [[  9.99679804e-01   3.20200255e-04]]
result: [[  9.99797165e-01   2.02902214e-04]]
result: [[  9.99907970e-01   9.19987069e-05]]
result: [[  9.99867797e-01   1.32186353e-04]]
result: [[  9.99911308e-01   8.87137867e-05]]
result: [[  9.99914169e-01   8.58106796e-05]]
result: [[  9.99934316e-01   6.56561315e-05]]
result: [[  9.99884129e-01   1.15893039e-04]]
result: [[  9.99933004e-01   6.696

result: [[  9.99848843e-01   1.51075918e-04]]
result: [[  9.99841332e-01   1.58674797e-04]]
result: [[  9.99872684e-01   1.27276930e-04]]
result: [[  9.99808490e-01   1.91554573e-04]]
result: [[  9.99809921e-01   1.90079736e-04]]
result: [[  9.99797523e-01   2.02438430e-04]]
result: [[  9.99794662e-01   2.05354751e-04]]
result: [[  9.99815285e-01   1.84745295e-04]]
result: [[  9.99828815e-01   1.71181484e-04]]
result: [[  9.99843955e-01   1.55968359e-04]]
result: [[  9.99832749e-01   1.67166654e-04]]
result: [[  9.99838471e-01   1.61518998e-04]]
result: [[  9.99838948e-01   1.60983298e-04]]
result: [[  9.99914289e-01   8.57589912e-05]]
result: [[  9.99770582e-01   2.29449171e-04]]
result: [[  9.99836564e-01   1.63427612e-04]]
result: [[  9.99759495e-01   2.40516485e-04]]
result: [[  9.99820769e-01   1.79261129e-04]]
result: [[  9.99811351e-01   1.88666454e-04]]
result: [[  9.99827206e-01   1.72877408e-04]]
result: [[  9.99847531e-01   1.52498658e-04]]
result: [[  9.99837756e-01   1.621

result: [[ 0.99565071  0.0043493 ]]
result: [[ 0.99454725  0.00545276]]
result: [[ 0.99528837  0.00471156]]
result: [[ 0.99481022  0.00518976]]
result: [[ 0.99446917  0.00553086]]
result: [[ 0.99451977  0.00548021]]
result: [[ 0.99327397  0.00672601]]
result: [[ 0.99351645  0.00648359]]
result: [[ 0.99215066  0.0078494 ]]
result: [[ 0.99257857  0.00742139]]
result: [[ 0.99032658  0.00967341]]
result: [[ 0.99072838  0.0092716 ]]
result: [[ 0.98923165  0.0107684 ]]
result: [[ 0.9895404   0.01045959]]
result: [[ 0.98813754  0.0118625 ]]
result: [[ 0.9894141   0.01058594]]
result: [[ 0.98868573  0.01131424]]
result: [[ 0.98077458  0.01922548]]
result: [[ 0.97822052  0.0217795 ]]
result: [[ 0.98624033  0.01375971]]
result: [[ 0.99045604  0.00954397]]
result: [[ 0.98978639  0.01021365]]
result: [[ 0.99315166  0.00684833]]
result: [[ 0.99304914  0.00695083]]
result: [[ 0.98062474  0.01937525]]
result: [[ 0.91837376  0.08162621]]
result: [[ 0.03133469  0.96866536]]
result: [[ 0.98817879  0.011

result: [[ 0.99795026  0.00204973]]
result: [[ 0.40505359  0.59494644]]
result: [[ 0.86079854  0.13920152]]
result: [[ 0.68000728  0.31999266]]
result: [[ 0.02142376  0.97857624]]
result: [[ 0.01730387  0.98269618]]
result: [[ 0.46546289  0.53453714]]
result: [[ 0.42080539  0.57919455]]
result: [[ 0.2117649   0.78823513]]
result: [[ 0.26298848  0.73701155]]
result: [[ 0.29351106  0.70648891]]
result: [[ 0.29730859  0.70269144]]
result: [[ 0.19428173  0.8057183 ]]
result: [[  1.57044940e-06   9.99998450e-01]]
result: [[  4.81553343e-06   9.99995232e-01]]
result: [[ 0.2191385   0.78086156]]
result: [[ 0.30973879  0.69026124]]
result: [[ 0.37144402  0.62855595]]
result: [[  9.99100924e-01   8.99046718e-04]]
result: [[ 0.98831439  0.01168558]]
result: [[ 0.78326315  0.21673688]]
result: [[ 0.65984446  0.34015548]]
result: [[ 0.42660722  0.57339281]]
result: [[ 0.52717841  0.47282165]]
result: [[ 0.27408803  0.72591197]]
result: [[ 0.42677817  0.5732218 ]]
result: [[ 0.25895852  0.74104148]

result: [[ 0.77890033  0.22109962]]
result: [[ 0.01689767  0.98310238]]
result: [[ 0.72524166  0.27475828]]
result: [[ 0.68984336  0.3101567 ]]
result: [[ 0.01417958  0.98582041]]
result: [[ 0.75343508  0.24656495]]
result: [[ 0.74563873  0.25436127]]
result: [[ 0.78814828  0.21185176]]
result: [[ 0.74815506  0.25184491]]
result: [[ 0.72219008  0.27780992]]
result: [[ 0.74425501  0.25574508]]
result: [[ 0.79450202  0.20549802]]
result: [[ 0.65380597  0.34619397]]
result: [[ 0.64499456  0.3550055 ]]
result: [[ 0.63330162  0.36669835]]
result: [[ 0.00511052  0.9948895 ]]
result: [[ 0.61159265  0.38840732]]
result: [[ 0.72658402  0.27341598]]
result: [[ 0.01090346  0.98909658]]
result: [[ 0.60712731  0.3928726 ]]
result: [[ 0.02191334  0.97808665]]
result: [[ 0.57171232  0.42828771]]
result: [[ 0.62438953  0.37561056]]
result: [[ 0.02499515  0.97500479]]
result: [[ 0.03728721  0.96271271]]
result: [[ 0.71050733  0.28949264]]
result: [[ 0.0399764   0.96002352]]
result: [[ 0.85487682  0.145